In [2]:
import os
import cv2
import PIL
import face_recognition
import numpy as np
from PIL import Image
from operator import itemgetter

In [4]:
def get_every_frame(video_path , interval=1):
    frames = []
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()

    count = 0
    while success:
        if not count % interval:
            frames.append(image[:, :, ::-1])
        success, image = vidcap.read()
        count += 1

    return frames

In [5]:
def get_face_locations(frames, zoomed=False):
    face_coordinates = []
    for frame in frames:
        coordinates_found = face_recognition.face_locations(frame)
        if coordinates_found is not None and len(coordinates_found):
            face_coordinates.append(coordinates_found[-1])
        else:
            face_coordinates.append(None)

    if not zoomed:
        cleaned_face_coordinates = [f for f in face_coordinates if f is not None]
        min_top = min(cleaned_face_coordinates, key=itemgetter(0))[0]
        max_right = max(cleaned_face_coordinates, key=itemgetter(1))[1]
        max_bottom = max(cleaned_face_coordinates, key=itemgetter(2))[2]
        min_left = min(cleaned_face_coordinates, key=itemgetter(3))[3]

        return min_top, max_right, max_bottom, min_left

In [6]:
def get_faces(frames, zoomed=False):
    face_coordinates = get_face_locations(frames)
    
    cleaned_face_coordinates = [f for f in face_coordinates if f is not None]
    min_top = min(cleaned_face_coordinates, key=itemgetter(0))[0]
    max_right = max(cleaned_face_coordinates, key=itemgetter(1))[1]
    max_bottom = max(cleaned_face_coordinates, key=itemgetter(2))[2]
    min_left = min(cleaned_face_coordinates, key=itemgetter(3))[3]

    faces = []
    if zoomed:
        for index, frame in enumerate(frames):
            if face_coordinates[index] is not None:
                top, right, bottom, left = face_coordinates[index]
                faces.append(frame[top:bottom, left:right])
    else:
        for frame in frames:
            if face_coordinates is not None:
                faces.append(frame[min_top:max_bottom, min_left:max_right])
                
    return faces

In [7]:
def average(frames):
    w, h = Image.fromarray(frames[0]).size
    N = len(frames)

    avg = numpy.zeros((h,w,3), numpy.float)

    for frame in frames:
        frame = numpy.array(frame, dtype=numpy.float)
        avg += (frame/N)

    avg = numpy.array(numpy.round(avg), dtype=numpy.uint8)

    return avg

In [101]:
def difference(frames, interval):
    w, h = Image.fromarray(frames[0]).size
    
    diff = numpy.zeros((h,w,3), numpy.float)

    for current_frame, next_frame in zip(frames[::interval], frames[interval::interval]):
        current_frame = numpy.array(current_frame, dtype=numpy.float)
        next_frame = numpy.array(next_frame, dtype=numpy.float)
        diff += np.absolute(next_frame - current_frame)

    diff = numpy.array(numpy.round(diff), dtype=numpy.uint8)

    return diff

In [ ]:
def stabilise(faces_locations):


In [102]:
video_dir_path = '.\Celeb-DF-v2\Celeb-real'
video_listdir = os.listdir(video_dir_path)
video = f'{video_dir_path}\{video_listdir[17]}'

In [ ]:
# get every frame
# looked at zoomed in faces (locations)
# stable locations and find frames that pass test
# find max box for those frames
# carry on

In [104]:
frames = get_every_frame(video, interval=1)
face_locations = get_face_locations(frames, zoomed=False)
stable_face_locations = get_stabilised_face_locations(faces, movement_thershold)
faces = get_faces(stable_face_locations)

# diff_face = difference(faces, interval=len(faces)//10)
# Image.fromarray(diff_face)

# avg_face = average(faces)

KeyboardInterrupt: 